In [1]:
!pip install boto3 langchain langchain_community -Uq

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-ai 2.8.1 requires faiss-cpu, which is not installed.
aiobotocore 2.7.0 requires botocore<1.31.65,>=1.31.16, but you have botocore 1.34.67 which is incompatible.
jupyter-ai 2.8.1 requires langchain==0.0.350, but you have langchain 0.1.13 which is incompatible.
jupyter-ai 2.8.1 requires langchain-core<0.1.4,>=0.1.0, but you have langchain-core 0.1.33 which is incompatible.
jupyter-ai-magics 2.8.1 requires langchain==0.0.350, but you have langchain 0.1.13 which is incompatible.
jupyter-ai-magics 2.8.1 requires langchain-core<0.1.4,>=0.1.0, but you have langchain-core 0.1.33 which is incompatible.


In [9]:
import json
import boto3
import random
import pandas as pd

In [2]:
bedrock_runtime = boto3.client('bedrock-runtime')

In [155]:
filename = '游戏聊天日志03.06~03.09.csv'
df = pd.read_csv(filename)
# with open(filename) as f:
#     lines = f.readlines()

In [156]:
df.shape

(1394177, 4)

In [157]:
df.channelid.describe()

count    1.394177e+06
mean     3.680513e+00
std      2.810867e+00
min      0.000000e+00
25%      0.000000e+00
50%      6.000000e+00
75%      6.000000e+00
max      6.000000e+00
Name: channelid, dtype: float64

In [158]:
df.groupby('channelid').count()

,@timestamp,senderrolename,_c3
channelid,,,
0,451691,451573,451691
1,86577,86577,86577
5,90744,90744,90744
6,765165,765165,765164


In [162]:
df = df.sort_values(by=['channelid', '@timestamp'], ascending=[True, True])

In [163]:
df

,@timestamp,channelid,senderrolename,_c3
1010571,2024-03-06 00:00:01.550,0,一碰就抖,帮会？
1010574,2024-03-06 00:00:01.844,0,孤影随形,没，没元宝
1010579,2024-03-06 00:00:02.802,0,桃丶夭,晚安
1010581,2024-03-06 00:00:03.622,0,日出,说个价，完事我就给你拆下来几个
1010583,2024-03-06 00:00:05.684,0,丶伊人,<器魂·10级>拆了昨天的9，又来个10
...,...,...,...,...
917530,2024-03-09 23:59:57.149,6,峥嵘岁月,那42的我试了，还不如低概率的更容易出
917527,2024-03-09 23:59:57.313,6,许丨丶三哥,我看别个主播换地图洗。还真被洗出来了
917528,2024-03-09 23:59:57.358,6,骆天宇,自爆掌毒才好，青衣反正攻高
917537,2024-03-09 23:59:57.933,6,叉叉,还在打啊


In [167]:
c_id = 0

In [187]:
%%time
df['chat'] = df.apply(lambda x: f"<channel>{'世界频道_'+str(x['channelid']) if x['channelid'] == 0 else '其他频道_'+str(x['channelid']) }<channel><username>{x['senderrolename']}</username><message>{x['_c3']}</message>", axis=1)

CPU times: user 16.9 s, sys: 67.3 ms, total: 16.9 s
Wall time: 16.9 s


In [188]:
df_new = df[df.channelid==c_id]
df_new

,@timestamp,channelid,senderrolename,_c3,chat
1010571,2024-03-06 00:00:01.550,0,一碰就抖,帮会？,<channel>世界频道_0<channel><username>一碰就抖</userna...
1010574,2024-03-06 00:00:01.844,0,孤影随形,没，没元宝,<channel>世界频道_0<channel><username>孤影随形</userna...
1010579,2024-03-06 00:00:02.802,0,桃丶夭,晚安,<channel>世界频道_0<channel><username>桃丶夭</usernam...
1010581,2024-03-06 00:00:03.622,0,日出,说个价，完事我就给你拆下来几个,<channel>世界频道_0<channel><username>日出</username...
1010583,2024-03-06 00:00:05.684,0,丶伊人,<器魂·10级>拆了昨天的9，又来个10,<channel>世界频道_0<channel><username>丶伊人</usernam...
...,...,...,...,...,...
917531,2024-03-09 23:59:56.698,0,流德很,一tiao龙,<channel>世界频道_0<channel><username>流德很</usernam...
917524,2024-03-09 23:59:56.947,0,狼烟四起战不休,好,<channel>世界频道_0<channel><username>狼烟四起战不休</use...
917532,2024-03-09 23:59:57.291,0,南东,在不,<channel>世界频道_0<channel><username>南东</username...
917535,2024-03-09 23:59:59.062,0,洪灬熙官,战忍 高密加的太多了 不止会心 还有忽略闪避 还加完全闪避内外功,<channel>世界频道_0<channel><username>洪灬熙官</userna...


In [189]:
extracted_data = list(df_new.chat.values)
len(extracted_data)

451691

In [171]:
from langchain_community.chat_models import BedrockChat
from langchain_core.messages import HumanMessage,AIMessage
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder,HumanMessagePromptTemplate

In [290]:
llm_sonnet = BedrockChat(model_id="anthropic.claude-3-sonnet-20240229-v1:0",
                  model_kwargs={"temperature": 0,
                                "top_k":10,
                                "max_tokens": 4000,
                                "top_p":0.5,
                                # "stop_sequences":['</response>']
                               },
                  streaming=True,callbacks=[StreamingStdOutCallbackHandler()])

llm_haiku = BedrockChat(model_id="anthropic.claude-3-haiku-20240307-v1:0",
                  model_kwargs={"temperature": 0,
                                "top_k":10,
                                "max_tokens": 4000,
                                "top_p":0.5,
                                # "stop_sequences":['</response>']
                               },
                  streaming=True,callbacks=[StreamingStdOutCallbackHandler()])

In [324]:
template_extract = \
"""You are a great linguist classifer, and now you are asked to analyze the group chat history of users in a game, and identify the user if it is against the rules.
The rules is defined as classes [A,B,C,D]
Below is the definition of classes:
- A: "小号脚本广告",特征有：小号高频发言、文案内容重复、点对点私聊拉人；表现形式有：私聊询问要不要做拖、或者高福利玩XXX游戏；私聊询问要不要福利；私聊免费送福利； 私聊要不要一起玩新游；私聊给激活码；
- B: "诱导加微拉人",特征有：小号在世界频道或私聊以各种理由，诱导玩家加微信或者加QQ；表现形式有：小号在世界频道重复发言，说有福利码，要的私聊，私聊后要求加微信或者加QQ才给福利码；小号在世界频道收月卡号、收金币，私聊后，要求加微或者加QQ详聊；小号在世界频道说找队友一起玩游戏或者说不会玩找人教，私聊小号后，小号要求加微或者加QQ；小号在世界频道出号、免费送号、送金币，私聊后要求加微或QQ详聊；小号改名为【花的票帮找回】【氪多帮追回】等名字，在世界频道重复发言，称充值可退款，私聊要求加微信或者加QQ；
- C: "诱导关注公众号拉人", 特征有：小号在世界频道或私聊诱导玩家关注公众号；表现形式有：小号在世界频道刷屏，说免费送礼包金币兑换码，诱导关注公众号；小号把名字改为带公众号的ID，说白嫖金币兑换码，诱导关注公众号；
- D: "工作室家族建群拉人；", 特征有：族长/管理/小号在家族/帮会频道高频发言、文案内容重复、私聊拉人. 表现形式:族长/管理在家族/帮会/私聊频道刷屏，说马上清理机器人/一起下副本/组固定队，要求玩家进群；族长/管理在家族/帮会频道刷屏，说让玩家注意查看邮件进群；族长不发言，家族内未充值小号在家族/帮会/私聊频道以族长名义拉人进群；

Here is the tasks:
Only find the quotes from the message that are against the rules and then print them in xml tag <response>. 
Quotes should be exactly the same as original text.
If there are no relevant quotes at all, write "No relevant quotes" instead.

Here is a chat history you will analyze
<doc>
{context}
</doc>


please enclose your analysis results in xml tag <response>.

the output format is json, for example:
<response>
[
{{"channel":xxx,"username":xxx,"message":xxx}}
]
</response>


Skip the preamble, go straight into the answer.
"""

In [325]:
template_sentiment = \
"""You are a INFORMATION classifier
Now you are asked to analyze the group chat history of users in a game, and identify the user if it is against the rules.
Give you a list of sentences you will classify them to the predefined classes [A,B,C,D,N], output the class in <label_class>. and output a confidence score between 0 to 10, that represents how relevant it is to the class label, 10 means most relevant, and 0 is not relevant
Below is the definition of classes:
- A: "小号脚本广告",特征有：小号高频发言、文案内容重复、点对点私聊拉人；表现形式有：私聊询问要不要做拖、或者高福利玩XXX游戏；私聊询问要不要福利；私聊免费送福利； 私聊要不要一起玩新游；私聊给激活码；
- B: "诱导加微拉人",特征有：小号在世界频道或私聊以各种理由，诱导玩家加微信或者加QQ；表现形式有：小号在世界频道重复发言，说有福利码，要的私聊，私聊后要求加微信或者加QQ才给福利码；小号在世界频道收月卡号、收金币，私聊后，要求加微或者加QQ详聊；小号在世界频道说找队友一起玩游戏或者说不会玩找人教，私聊小号后，小号要求加微或者加QQ；小号在世界频道出号、免费送号、送金币，私聊后要求加微或QQ详聊；小号改名为【花的票帮找回】【氪多帮追回】等名字，在世界频道重复发言，称充值可退款，私聊要求加微信或者加QQ；
- C: "诱导关注公众号拉人", 特征有：小号在世界频道或私聊诱导玩家关注公众号；表现形式有：小号在世界频道刷屏，说免费送礼包金币兑换码，诱导关注公众号；小号把名字改为带公众号的ID，说白嫖金币兑换码，诱导关注公众号；
- D: "工作室家族建群拉人；", 特征有：族长/管理/小号在家族/帮会频道高频发言、文案内容重复、私聊拉人. 表现形式:族长/管理在家族/帮会/私聊频道刷屏，说马上清理机器人/一起下副本/组固定队，要求玩家进群；族长/管理在家族/帮会频道刷屏，说让玩家注意查看邮件进群；族长不发言，家族内未充值小号在家族/帮会/私聊频道以族长名义拉人进群；
- E: "其他"， 不符合A,B,C,D分类特征。

Here are the sentences:
<sentences>
{relevant_info}
</sentences>

Please follow below requirements:
1. You will strictly be based on the document in <doc>.
2. please enclose your analysis results in xml tag <output>.
3. output a score between 0 -10,

the output format for example:
<output>
<channel>xxx</channel><username>xxx</username><message>xxx</message><label_class>A</label_class><score>10<score>
</output>

Skip the preamble, go straight into the answer. 

"""

In [326]:
from langchain_core.output_parsers.base import BaseOutputParser
import re
class CustOuputParser(BaseOutputParser[str]):

    def extract(self,content:str) -> str:
        pattern = r"<response>(.*?)</response>"
        match = re.search(pattern, content, re.DOTALL)
        if match:
            text = match.group(1)
            text = text.replace('[','(').replace(']',')') ##避免跟sentiment的格式冲突
            return text
        else:
            return 'No relevant quotes'    
    
    def parse(self, text: str) -> str:
        cleaned_text = self.extract(text)
        return cleaned_text

    @property
    def _type(self) -> str:
        return "cust_output_parser"


class CustOuputParser2(BaseOutputParser[str]):

    def extract(self,content:str) -> str:
        pattern = r"<output>(.*?)</output>"
        match = re.search(pattern, content, re.DOTALL)
        return match.group(1) if match else 'No sentiment'    
    
    def parse(self, text: str) -> str:
        cleaned_text = self.extract(text)
        return cleaned_text

    @property
    def _type(self) -> str:
        return "cust_output_parser"
    

In [327]:
def split_into_chunks(arr, n):
    chunks = []
    for i in range(0, len(arr), n):
        chunks.append(arr[i:i+n])
    return chunks

In [328]:
chunks = split_into_chunks(extracted_data,1000)

In [329]:
print(f"total chunks {len(chunks)}")

total chunks 452


In [330]:
text =  "\n".join(chunks[0])
len(text)

79617

In [331]:
# from langchain_core.output_parsers import StrOutputParser
# prompt = ChatPromptTemplate.from_template(template)
prompt_extract = ChatPromptTemplate.from_template(template_extract)
prompt_sentiment = ChatPromptTemplate.from_template(template_sentiment)

In [332]:
from langchain_core.runnables import RunnableLambda
from operator import itemgetter
## 如果没有相关内容则无须invoke chain2
def route(info):
    if not 'no relevant quotes' in info['relevant_info'].lower():
        return chain_2

In [333]:
output_parser = CustOuputParser()
output2_parser  = CustOuputParser2()

chain_1 = prompt_extract | llm_sonnet | output_parser
chain_2 = prompt_sentiment | llm_sonnet| output2_parser
# chain_3 = ({'relevant_info':chain_1,'topic':itemgetter('topic')})|prompt_sentiment | llm_haiku|output2_parser


In [334]:
##测试chain3
# chain_3.invoke({'topic':"auction house",
#                        'context':text})

In [335]:
%%time
##测试加入了路由选择的chainfull
full_chain = ({'relevant_info':chain_1})| RunnableLambda(route)
answer = full_chain.invoke({'context':text})

<response>
[
{"channel":"世界频道_0","username":"一剑纵横天下","message":"好了好了，我加你微信了，你同意一下是不是刘磊对不对。"},
{"channel":"世界频道_0","username":"一剑纵横天下","message":"我加你了，同意一下。"},
{"channel":"世界频道_0","username":"一剑纵横天下","message":"刘磊是吧"},
{"channel":"世界频道_0","username":"沸羊羊","message":"严哥，兄弟，我是咱们家族组长，你能听到能看到的家族回一下，我在家族平台喊你了。"},
{"channel":"世界频道_0","username":"沸羊羊","message":"顺丰兄弟啊，我是咱们家族族长，我在家族平台喊你呢，你能听到能看到的家族回一下。"},
{"channel":"世界频道_0","username":"沸羊羊","message":"L0004956"}
]
</response><output>
<channel>世界频道_0</channel><username>一剑纵横天下</username><message>好了好了，我加你微信了，你同意一下是不是刘磊对不对。</message><label_class>B</label_class><score>9</score>
<channel>世界频道_0</channel><username>一剑纵横天下</username><message>我加你了，同意一下。</message><label_class>B</label_class><score>9</score>
<channel>世界频道_0</channel><username>一剑纵横天下</username><message>刘磊是吧</message><label_class>B</label_class><score>9</score>
<channel>世界频道_0</channel><username>沸羊羊</username><message>严哥，兄弟，我是咱们家族组长，你能听到能看到的家族回一下，我在家族平台喊你了。</message><label_class>D</label_cl

In [337]:
print(answer)


<channel>世界频道_0</channel><username>一剑纵横天下</username><message>好了好了，我加你微信了，你同意一下是不是刘磊对不对。</message><label_class>B</label_class><score>9</score>
<channel>世界频道_0</channel><username>一剑纵横天下</username><message>我加你了，同意一下。</message><label_class>B</label_class><score>9</score>
<channel>世界频道_0</channel><username>一剑纵横天下</username><message>刘磊是吧</message><label_class>B</label_class><score>9</score>
<channel>世界频道_0</channel><username>沸羊羊</username><message>严哥，兄弟，我是咱们家族组长，你能听到能看到的家族回一下，我在家族平台喊你了。</message><label_class>D</label_class><score>8</score>
<channel>世界频道_0</channel><username>沸羊羊</username><message>顺丰兄弟啊，我是咱们家族族长，我在家族平台喊你呢，你能听到能看到的家族回一下。</message><label_class>D</label_class><score>8</score>
<channel>世界频道_0</channel><username>沸羊羊</username><message>L0004956</message><label_class>N</label_class><score>2</score>



In [275]:
# question = "is there any content relevant to auction house?"

#'please list all the content if it is relevant and classify the sentiment of each content into [positive,neutral,negative]'

# answer = chain_2.invoke({'topic':"auction house",
#                        'context':text})

In [338]:
def extract_info(text):
    pattern = r'<channel>(.*?)</channel><username>(.*?)</username><message>(.*?)</message><label_class>(.*?)</label_class><score>(.*?)</score>'
    matches = re.findall(pattern, text)
    print(matches)
    result = []
    for match in matches:
        # info = {
        #     'channel': match[0],
        #     'username': match[1],
        #     'message': match[2],
        #     'label_class': match[3],
        #     'score': int(match[4])
        # }
        info = [match[0], match[1], match[2],  match[3], int(match[4])]
        result.append(info)
    return result

In [339]:
ret= extract_info(answer)
ret

[('世界频道_0', '一剑纵横天下', '好了好了，我加你微信了，你同意一下是不是刘磊对不对。', 'B', '9'), ('世界频道_0', '一剑纵横天下', '我加你了，同意一下。', 'B', '9'), ('世界频道_0', '一剑纵横天下', '刘磊是吧', 'B', '9'), ('世界频道_0', '沸羊羊', '严哥，兄弟，我是咱们家族组长，你能听到能看到的家族回一下，我在家族平台喊你了。', 'D', '8'), ('世界频道_0', '沸羊羊', '顺丰兄弟啊，我是咱们家族族长，我在家族平台喊你呢，你能听到能看到的家族回一下。', 'D', '8'), ('世界频道_0', '沸羊羊', 'L0004956', 'N', '2')]


[['世界频道_0', '一剑纵横天下', '好了好了，我加你微信了，你同意一下是不是刘磊对不对。', 'B', 9],
 ['世界频道_0', '一剑纵横天下', '我加你了，同意一下。', 'B', 9],
 ['世界频道_0', '一剑纵横天下', '刘磊是吧', 'B', 9],
 ['世界频道_0', '沸羊羊', '严哥，兄弟，我是咱们家族组长，你能听到能看到的家族回一下，我在家族平台喊你了。', 'D', 8],
 ['世界频道_0', '沸羊羊', '顺丰兄弟啊，我是咱们家族族长，我在家族平台喊你呢，你能听到能看到的家族回一下。', 'D', 8],
 ['世界频道_0', '沸羊羊', 'L0004956', 'N', 2]]

In [340]:
#save a list to a local file, if the file is exist then append the list to the file
def save_list(list,file_name):
    with open(file_name,'a') as f:
        for item in list:
            f.write("%s\n" % item)
    f.close()
    return True

In [341]:
#save a list to a csv file using pandas, if the file is exist then append the list to the file
import pandas as pd
import os
def save_to_csv(data, filename):
    new_data = pd.DataFrame(data, columns=['channel', 'username', 'message', 'label_class', 'score'])
    if os.path.isfile(filename):
        df = pd.read_csv(filename)
        #append data to df
        df = pd.concat([df, new_data], ignore_index=True)
    else:
        df = new_data
    df.to_csv(filename, index=False)
    return df

循环每个chunk进行输出,每次追加到csv文件保存，

In [345]:
import time
all_result = []
filename = f'chlid_{c_id}_parsed_result'

##断点跳过
skip_num =9
for i,chunk in enumerate(chunks):
    if i < skip_num:
        continue
    t1 = time.time()
    print(f'--------chunk idx:{i}-------')
    text =  "\n".join(chunk)
    answer = full_chain.invoke({
                       'context': text})
    extract_ret = extract_info(answer)
    print(f'\nextract_ret:\n{extract_ret}')
    time.sleep(40)
    print(f'---time cost {time.time()-t1} s -----\nsleep 10s for token tpm\n\n')
    if extract_ret:
        all_result+=extract_ret
        save_to_csv(extract_ret,filename+f"chunk_{i}.csv")
        save_to_csv(extract_ret,filename+f"_all.csv")

--------chunk idx:9-------
<response>
[
{"channel":"世界频道_0","username":"芈月","message":"不用绑定，关注微 信公 众 号，对话框发送zwj999给送主 修 令"},
{"channel":"世界频道_0","username":"张淮阳","message":"发ZWJ999"},
{"channel":"世界频道_0","username":"九合妖域","message":"手机微信公众号里关注剑侠世界起源 打开键盘输入zwj999"},
{"channel":"世界频道_0","username":"繁华丶落雨","message":"去微信公众号里发ZWJ999"},
{"channel":"世界频道_0","username":"暴走学院","message":"你在公众号发 ZJ999"},
{"channel":"世界频道_0","username":"暴走学院","message":"在微信里找剑侠起源的公众号"},
{"channel":"世界频道_0","username":"暴走学院","message":"ZWJ999"}
]
</response><output>
<channel>世界频道_0</channel><username>芈月</username><message>不用绑定，关注微 信公 众 号，对话框发送zwj999给送主 修 令</message><label_class>C</label_class><score>10</score>
<channel>世界频道_0</channel><username>张淮阳</username><message>发ZWJ999</message><label_class>C</label_class><score>10</score>
<channel>世界频道_0</channel><username>九合妖域</username><message>手机微信公众号里关注剑侠世界起源 打开键盘输入zwj999</message><label_class>C</label_class><score>10</score>
<channel>世界频道_0</channel><username>繁华丶落雨</u

EventStreamError: An error occurred (throttlingException) when calling the InvokeModelWithResponseStream operation: Too many requests, please wait before trying again. You have sent too many requests.  Wait before trying again.

In [144]:
len(all_result)

92